# Note

This is a scribble notebook for me to gain insights from the given data. It is strongly recommended to read the comments to understand what the code is doing and the assumptions I made based on the insights.

# Load Data & Pre-processing

In [1]:
import pandas as pd
from datetime import date, datetime


In [2]:
employees_df = pd.read_csv("employees.csv")
employees_df = employees_df.rename(columns={'employe_id': 'employee_id'})
timesheets_df = pd.read_csv("timesheets.csv")

In [3]:
# Merge the 'employees' and 'timesheets' data based on employee_id
df = pd.merge(timesheets_df, employees_df, on='employee_id')
df.head()

,timesheet_id,employee_id,date,checkin,checkout,branch_id,salary,join_date,resign_date
0,23907432,66,2019-08-21,08:13:31,17:05:02,2,10000000,2018-12-10,NaN
1,23907443,66,2019-08-22,08:28:27,17:20:25,2,10000000,2018-12-10,NaN
2,23907452,66,2019-08-26,07:55:59,17:16:47,2,10000000,2018-12-10,NaN
3,23907461,66,2019-08-27,07:59:01,17:00:47,2,10000000,2018-12-10,NaN
4,23907472,66,2019-08-28,08:14:13,18:13:51,2,10000000,2018-12-10,NaN


In [4]:
df = pd.merge(timesheets_df, employees_df, on='employee_id')
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

max_date = df['date'].max() # Getting new data
filtered_df = df[df['date'].dt.month == max_date.month] # Get all data from the same month as the new data

filtered_df['checkin'] = pd.to_datetime(filtered_df['date'].astype(str) + ' ' + filtered_df['checkin'])
filtered_df['checkout'] = pd.to_datetime(filtered_df['date'].astype(str) + ' ' + filtered_df['checkout'])

filtered_df['hours_worked'] = (filtered_df['checkout'] - filtered_df['checkin']).dt.total_seconds() / 3600
filtered_df['hours_worked'] = filtered_df['hours_worked'].abs()

df = filtered_df
df.head()

/var/folders/6z/n5n0hdl93rz8v87bx7nynpg40000gn/T/ipykernel_85581/58445521.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['checkin'] = pd.to_datetime(filtered_df['date'].astype(str) + ' ' + filtered_df['checkin'])
/var/folders/6z/n5n0hdl93rz8v87bx7nynpg40000gn/T/ipykernel_85581/58445521.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['checkout'] = pd.to_datetime(filtered_df['date'].astype(str) + ' ' + filtered_df['checkout'])
/var/folders/6z/n5n0hdl93rz8v87bx7nynpg40000gn/T/

,timesheet_id,employee_id,date,checkin,checkout,branch_id,salary,join_date,resign_date,month,year,hours_worked
65,30014919,66,2019-12-02,2019-12-02 08:43:16,2019-12-02 19:22:29,2,10000000,2018-12-10,NaN,12,2019,10.653611
66,30150956,66,2019-12-03,2019-12-03 08:17:50,2019-12-03 17:46:19,2,10000000,2018-12-10,NaN,12,2019,9.474722
67,30247787,66,2019-12-04,2019-12-04 08:55:02,2019-12-04 18:52:56,2,10000000,2018-12-10,NaN,12,2019,9.965000
68,30320933,66,2019-12-05,2019-12-05 08:43:48,2019-12-05 18:10:04,2,10000000,2018-12-10,NaN,12,2019,9.437778
69,30397129,66,2019-12-06,2019-12-06 08:48:29,2019-12-06 17:51:54,2,10000000,2018-12-10,NaN,12,2019,9.056944


In [5]:
# Check if I can do something with resign_date
df[df['resign_date'].notna()].groupby(['employee_id', 'branch_id']).size().reset_index(name='count').head()

,employee_id,branch_id,count
0,9,1,16
1,10,1,20
2,17,1,16
3,20,1,18
4,29,1,15


**Conclusion**: assume resign date doesn't affect the employee's salary when they resign

# Anomaly 1: `hours_worked` is equal to 0
- Assume: `hours_worked` value of 0 indicates people on paid leave, an alternative approach is to replace it with NaN, which will be processed further. But in this case I will stick to my assumption of paid-leave so I'll keep the 0 values.

In [6]:
df[df['hours_worked']==0].head()

,timesheet_id,employee_id,date,checkin,checkout,branch_id,salary,join_date,resign_date,month,year,hours_worked
30501,69808406,18,2020-12-17,2020-12-17 10:00:00,2020-12-17 10:00:00,1,12000000,2018-08-14,NaN,12,2020,0.0


# Anomaly 2: NaN values in `hours_worked`

**Cause**: there may be some NaN values on checkin/checkout.

In [7]:
df[df['hours_worked'].isna()].head()

,timesheet_id,employee_id,date,checkin,checkout,branch_id,salary,join_date,resign_date,month,year,hours_worked
289,68418777,66,2020-12-03,NaT,2020-12-03 18:00:00,2,10000000,2018-12-10,NaN,12,2020,NaN
290,68702419,66,2020-12-08,2020-12-08 08:22:00,NaT,2,10000000,2018-12-10,NaN,12,2020,NaN
296,69290255,66,2020-12-14,2020-12-14 08:30:00,NaT,2,10000000,2018-12-10,NaN,12,2020,NaN
299,69808500,66,2020-12-17,2020-12-17 08:44:00,NaT,2,10000000,2018-12-10,NaN,12,2020,NaN
301,70186999,66,2020-12-21,2020-12-21 06:57:00,NaT,2,10000000,2018-12-10,NaN,12,2020,NaN


**Altervative 1**: replace the missing data with average work hours (per month, per year, per branch_id) <br>
**Alternative 2**: replace `hours_worked` NaN values with 8 working hours, in accordance with the assumed company's policy.

I will use **Alternative 2** for the final transform code in the python file because it's more practical and reasonable. If we use **Alternative 1**, there might be missing `hours_worked` data from the begining of the month and we will be unable to get the average `hours_worked` in that month. 

### **Alternative 1** replace the missing data with average work hours

In [8]:
# Calculate average_work_hours by year, month, and branch_id
average_work_hours = df.dropna(subset=['hours_worked']).groupby(['year', 'month', 'branch_id'])['hours_worked'].mean().reset_index()
average_work_hours.rename(columns={'hours_worked': 'average_hours_perday'}, inplace=True)
average_work_hours.head()

,year,month,branch_id,average_hours_perday
0,2019,12,1,9.612864
1,2019,12,2,9.261725
2,2019,12,3,9.313454
3,2019,12,2590,8.047442
4,2019,12,2623,8.594911


In [9]:
# Create a mapping of 'year,' 'month,' and 'branch' to 'average_hours_perday'
average_hours_mapping = average_work_hours.set_index(['year', 'month', 'branch_id'])['average_hours_perday']
al1 = df
# Replace NaN values in df['hours_worked'] with values from 'average_work_hours' based on matching 'year,' 'month,' and 'branch'
al1.loc[al1['hours_worked'].isna(), 'hours_worked'] = al1.apply(lambda row: average_hours_mapping.get((row['year'], row['month'], row['branch_id']), row['hours_worked']), axis=1)

In [10]:
al1[al1['hours_worked'].isna()]

,timesheet_id,employee_id,date,checkin,checkout,branch_id,salary,join_date,resign_date,month,year,hours_worked


- Still getting missing data in 'hours_worked'. Because there maybe a date which every entry (based on year, month, branch_id) in timesheets has NaN value in 'checkin' or 'checkout'

In [11]:
average_work_hours.describe()

,year,month,branch_id,average_hours_perday
count,30.000000,30.0,30.000000,30.000000
mean,2019.500000,12.0,3092.766667,8.466221
std,0.508548,0.0,3288.979652,0.888846
min,2019.000000,12.0,1.000000,7.227292
25%,2019.000000,12.0,2598.250000,7.666070
50%,2019.500000,12.0,2629.000000,8.392593
75%,2020.000000,12.0,2633.750000,9.279868
max,2020.000000,12.0,12722.000000,10.287491


In [12]:
skewness = average_work_hours['average_hours_perday'].skew()
skewness

0.3405779818945454

Since the data is not too skewed, we can replace those missing datas in 'hours_worked' with the average of 'whole average_hours_perday'

**Assume**: our data is always not too skewed 

In [13]:
# Replace remaining NaN values in 'df'['hours_worked'] with values 
# from 'average_work_hours' (based on year, month, branch_id)
al1.loc[al1['hours_worked'].isna(), 'hours_worked'] = average_work_hours['average_hours_perday'].mean()

### **Alternative 2**: replace `hours_worked` NaN values with 8

In [14]:
df.loc[df['hours_worked'].isna(), 'hours_worked'] = 8

## `salary_per_hour` calculation

Finally, `hours_worked` column already cleaned. Now working on the `salary_per_hour` calculation

In [15]:
# DataFrame for total salary per month
total_salary_permonth = df.drop_duplicates(subset=['branch_id', 'year', 'month', 'employee_id']).groupby(['branch_id', 'year', 'month']).agg(
    total_salary=('salary', 'sum'),
    employee_counts=('employee_id', 'nunique')
).reset_index()
# DataFrame for sum of hours worked per month
sum_hours_worked = df.groupby(['branch_id', 'year', 'month']).agg(
    total_hours_worked=('hours_worked', 'sum'),
    employee_counts=('employee_id', 'nunique')
).reset_index()

result_df = total_salary_permonth.merge(sum_hours_worked, on=['branch_id', 'year', 'month', 'employee_counts'])
result_df['salary_per_hour'] = result_df['total_salary'] / result_df['total_hours_worked']
result_df['updated_at'] = datetime.now()
result_df.tail(5)

,branch_id,year,month,total_salary,employee_counts,total_hours_worked,salary_per_hour,updated_at
25,3092,2019,12,93000000,11,2164.909573,42957.914335,2023-10-23 01:32:32.145981
26,3092,2020,12,93000000,11,802.794672,115845.312916,2023-10-23 01:32:32.145981
27,11265,2020,12,11000000,1,164.733935,66774.341229,2023-10-23 01:32:32.145981
28,12722,2019,12,7000000,1,183.838333,38076.933538,2023-10-23 01:32:32.145981
29,12722,2020,12,7000000,1,96.059167,72871.754387,2023-10-23 01:32:32.145981


### Load the data to destination csv
Append new data to destination csv

In [16]:
try:
        existing_data = pd.read_csv('salary_per_hour.csv')
        existing_data = pd.concat([existing_data, result_df], ignore_index=True)
        existing_data.to_csv('salary_per_hour.csv', index=False)
except FileNotFoundError:
        # If the CSV file does not exist, create a new DataFrame from result_df
        existing_data = result_df
        existing_data.to_csv('salary_per_hour.csv', index=False)

existing_data.tail(5)

,branch_id,year,month,total_salary,employee_counts,total_hours_worked,salary_per_hour,updated_at
25,3092,2019,12,93000000,11,2164.909573,42957.914335,2023-10-23 01:32:32.145981
26,3092,2020,12,93000000,11,802.794672,115845.312916,2023-10-23 01:32:32.145981
27,11265,2020,12,11000000,1,164.733935,66774.341229,2023-10-23 01:32:32.145981
28,12722,2019,12,7000000,1,183.838333,38076.933538,2023-10-23 01:32:32.145981
29,12722,2020,12,7000000,1,96.059167,72871.754387,2023-10-23 01:32:32.145981
